In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

XPATH = By.XPATH

url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'


In [69]:
class Obligations_parser:
    def __init__(self, driver, new_session = True):
        self.driver = driver
        self.driver.maximize_window()
        self.prev_window_switch = 0

        if new_session:
            self.driver.get(url)
            
            
        # self.obligations_data = pd.DataFrame(columns = ['Название', "Код", 'Ссылка', "Срок(месяца)", "Цена", "Номинал", "Выплат в год", "Величина купона(-13%)", "Прибыль в год(-13% с купона)", "Доходность(все время)", "Доходность(год)", "Амортизация", 'Рейтинг'])
        self.obligations_data = pd.DataFrame()
    

    def next_layer(self, current_window_id : str):
        next_page_class = 'Pagination-module__item_ALrvz Pagination-module__item_size_s_voSgR Pagination-module__item_arrow_mCpmW'
        # retry_counter = 0
        
        try:
            self.driver.switch_to.window(current_window_id)

            buttons = self.driver.find_elements(XPATH, f'//a[@class = "{next_page_class}"]')

            if len(buttons) == 1:
                if self.prev_window_switch:
                    print('Все страницы пройдены.')
                    return(self.obligations_data)
                
                else:
                    self.prev_window_switch = 1
                    buttons[0].click()
            
            else:
                buttons[-1].click()
            



            print('Вызвали оконный итератор')
            self.elements_iterator()

        except Exception as e:
            
            print(str(e))
            # self.elements_iterator()
            return(self.obligations_data)
        


    
    def grab_paper_info(self, paper_url : str = ''):
        if paper_url:
            self.driver.execute_script("window.open('');")
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.driver.get(paper_url)
        
        else:
            self.driver.get(paper_url)

        paper_data = {}

        try:
            paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
            
            paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})


            paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
            paper_time = paper_time[paper_time.index('з') + 3 : ]
            paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})
            del paper_time

            paper_data.update({'Цена' : [float(self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
            
            paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

            paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')



            for counter in range(1, len(paper_datas) + 1, 2):
                match paper_datas[counter - 1].text:
                    case "Величина купона":
                        paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                        paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                        paper_data.update({'Величина купона(-13%)' : [paper_coupon]})
                    
                    case "Номинал":
                        paper_data.update({'Номинал' : [float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
                    
                    case "Количество выплат в год":
                        if 'Величина купона(-13%)' not in paper_data:
                            paper_data.update({'Величина купона(-13%)' : [0]})
                            
                        paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                        paper_data.update({'Доходность(год)' : [paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                        paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] / paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                    
                    case 'Амортизация':
                        paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                    

            print(paper_data)

            paper_data.update({'Ссылка' : [paper_url]})

            

            # print(pd.DataFrame(paper_data))
            # print(pd.DataFrame.from_dict(paper_data), )

            

            self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

            
            self.driver.close()

        except Exception:
            self.driver.close()
            time.sleep(10)
            self.grab_paper_info()


    def elements_iterator(self):

        # link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            
            self.grab_paper_info(paper_link)
            print(len(self.obligations_data), '\n')


        self.next_layer(self.driver.window_handles[0])
            

    
        # return(self.obligations_data)

            




        # for paper in papers_rows:
        #     paper = paper.find_element(XPATH, '//td').find_element(XPATH, f'//a[@class = "{link_buttons_class}"]')
        #     print(paper.get_attribute('href'))




In [70]:
# a = webdriver.Chrome()
a = webdriver.Firefox()


In [71]:
parser_object = Obligations_parser(a, True)

In [72]:
parser_object.elements_iterator()

{'Название': ['Феррони БО-01'], 'Код': ['RU000A1053R3'], 'Срок(месяца)': [21], 'Цена': [912.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [11.745], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [140.94], 'Доходность(все время)': [20.553749999999997]}
1 

{'Название': ['Роснано БО-002P выпуск 5'], 'Код': ['RU000A102HB3'], 'Срок(месяца)': [2], 'Цена': [882.0], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [29.232], 'Номинал': [1000.0], 'Выплат в год': [2], 'Доходность(год)': [58.464], 'Доходность(все время)': [29.232]}
2 

{'Название': ['Роснано выпуск 8'], 'Код': ['RU000A1008V9'], 'Срок(месяца)': [53], 'Цена': [831.2], 'Рейтинг': ['Средний'], 'Величина купона(-13%)': [58.203], 'Номинал': [1000.0], 'Выплат в год': [2], 'Доходность(год)': [116.406], 'Доходность(все время)': [1542.3795]}
3 

{'Название': ['Роснано БО-002P выпуск 4'], 'Код': ['RU000A101KK0'], 'Срок(месяца)': [5], 'Цена': [840.1], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [14.79], 'Номинал': [10

KeyboardInterrupt: 

In [68]:
parser_object.driver.window_handles

['ca99fa70-6ce5-46ce-9490-1a92c3699cd8']

Ошибка     
49 paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
---> 50 paper_time = paper_time[paper_time.index('з') + 3 : ]
     51 paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})

ValueError: substring not found

на https://www.tinkoff.ru/invest/bonds/RU000A106NT4/

In [10]:
df = parser_object.obligations_data.reset_index().drop('index', axis = 1)

In [11]:
df

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Ferroni выпуск 1,RU000A103XP8,24,750.0,Низкий,9.222,1000.0,12.0,110.664,221.328,https://www.tinkoff.ru/invest/bonds/RU000A103XP8/
1,Феррони БО-01,RU000A1053R3,22,933.2,Низкий,11.745,1000.0,12.0,140.940,258.390,https://www.tinkoff.ru/invest/bonds/RU000A1053R3/
2,СПМК БО-02,RU000A106FN3,5,966.8,Низкий,30.885,1000.0,4.0,123.540,463.275,https://www.tinkoff.ru/invest/bonds/RU000A106FN3/
3,Агрофирма-племзавод Победа выпуск 1,RU000A102S23,4,939.3,Низкий,21.141,1000.0,4.0,84.564,253.692,https://www.tinkoff.ru/invest/bonds/RU000A102S23/
4,Alfa Bank,RU000A100GW5,11,8350000.0,Средний,254862.150,10000000.0,4.0,1019448.600,8410450.950,https://www.tinkoff.ru/invest/bonds/RU000A100GW5/
...,...,...,...,...,...,...,...,...,...,...,...
809,РЖД 1Р-26R,RU000A106K43,57,1011.4,Высокий,9.396,1000.0,12.0,112.752,535.572,https://www.tinkoff.ru/invest/bonds/RU000A106K43/
810,Альфа-Банк 002Р выпуск 14,RU000A103RS4,2,1002.0,Высокий,34.452,1000.0,2.0,68.904,413.424,https://www.tinkoff.ru/invest/bonds/RU000A103RS4/
811,Сибнефтехимтрейд выпуск 2,RU000A104DZ7,4,1013.9,Низкий,13.224,1000.0,12.0,158.688,52.896,https://www.tinkoff.ru/invest/bonds/RU000A104DZ7/
812,Истрсыр БО,RU000A105ZF9,2,1051.0,Низкий,12.789,1000.0,12.0,153.468,25.578,https://www.tinkoff.ru/invest/bonds/RU000A105ZF9/


In [13]:
df.to_csv('Obligations_data_parsed.csv')

In [30]:
df[df['Номинал'] <= 15000].sort_values(by=['Доходность(год)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
290,ПЮДМ БО-П2,RU000A1020K7,21,10037.000000,Средний,92.916,10000.00,12.0,1114.992,1951.236,https://www.tinkoff.ru/invest/bonds/RU000A1020K7/
533,Нафтатранс Плюс выпуск 1,RU000A100303,3,10012.000000,Средний,84.303,10000.00,12.0,1011.636,252.909,https://www.tinkoff.ru/invest/bonds/RU000A100303/
678,Нафтатранс Плюс выпуск 2,RU000A100YD8,11,4664.466400,Низкий,49.590,4664.00,12.0,595.080,545.490,https://www.tinkoff.ru/invest/bonds/RU000A100YD8/
800,Селигдар Золотые облигации,RU000A1062M5,54,5609.824379,Высокий,70.992,5913.17,4.0,283.968,11500.704,https://www.tinkoff.ru/invest/bonds/RU000A1062M5/
130,АПРИ Флай Плэнинг,RU000A105DS9,37,1009.700000,Средний,52.026,1000.00,4.0,208.104,5774.886,https://www.tinkoff.ru/invest/bonds/RU000A105DS9/
...,...,...,...,...,...,...,...,...,...,...,...
22,Пионер-Лизинг-БО-П02,RU000A1006C3,63,1063.100000,Низкий,0.870,1000.00,12.0,10.440,54.810,https://www.tinkoff.ru/invest/bonds/RU000A1006C3/
787,ДОМ.РФ ИА 002P,RU000A0ZYJT2,271,80.373657,Средний,2.001,81.03,4.0,8.004,1626.813,https://www.tinkoff.ru/invest/bonds/RU000A0ZYJT2/
714,Ненецкий АО выпуск 1,RU000A0ZYF61,13,98.220000,Высокий,1.653,100.00,4.0,6.612,64.467,https://www.tinkoff.ru/invest/bonds/RU000A0ZYF61/
336,Верхнебаканский цементный завод 001P выпуск 1,RU000A0ZYLK7,50,973.900000,Средний,1.566,1000.00,4.0,6.264,234.900,https://www.tinkoff.ru/invest/bonds/RU000A0ZYLK7/


In [ ]:
df[df['Код'] == 'RU000A106WZ2']

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,102.0,Средний,45.501,1000.0,4,182.004,500.511,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/


In [22]:
df = pd.read_csv('Obligations_data_parsed.csv').drop('Unnamed: 0', axis = 1).fillna(0)
df = df[df['Доходность(все время)'] > 0]

In [25]:
df

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
0,Ferroni выпуск 1,RU000A103XP8,24,750.0,Низкий,9.222,1000.0,12.0,110.664,221.328,https://www.tinkoff.ru/invest/bonds/RU000A103XP8/
1,Феррони БО-01,RU000A1053R3,22,933.2,Низкий,11.745,1000.0,12.0,140.940,258.390,https://www.tinkoff.ru/invest/bonds/RU000A1053R3/
2,СПМК БО-02,RU000A106FN3,5,966.8,Низкий,30.885,1000.0,4.0,123.540,463.275,https://www.tinkoff.ru/invest/bonds/RU000A106FN3/
3,Агрофирма-племзавод Победа выпуск 1,RU000A102S23,4,939.3,Низкий,21.141,1000.0,4.0,84.564,253.692,https://www.tinkoff.ru/invest/bonds/RU000A102S23/
4,Alfa Bank,RU000A100GW5,11,8350000.0,Средний,254862.150,10000000.0,4.0,1019448.600,8410450.950,https://www.tinkoff.ru/invest/bonds/RU000A100GW5/
...,...,...,...,...,...,...,...,...,...,...,...
809,РЖД 1Р-26R,RU000A106K43,57,1011.4,Высокий,9.396,1000.0,12.0,112.752,535.572,https://www.tinkoff.ru/invest/bonds/RU000A106K43/
810,Альфа-Банк 002Р выпуск 14,RU000A103RS4,2,1002.0,Высокий,34.452,1000.0,2.0,68.904,413.424,https://www.tinkoff.ru/invest/bonds/RU000A103RS4/
811,Сибнефтехимтрейд выпуск 2,RU000A104DZ7,4,1013.9,Низкий,13.224,1000.0,12.0,158.688,52.896,https://www.tinkoff.ru/invest/bonds/RU000A104DZ7/
812,Истрсыр БО,RU000A105ZF9,2,1051.0,Низкий,12.789,1000.0,12.0,153.468,25.578,https://www.tinkoff.ru/invest/bonds/RU000A105ZF9/


In [27]:
df[df['Номинал'] <= 15000].sort_values(['Величина купона(-13%)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
290,ПЮДМ БО-П2,RU000A1020K7,21,10037.000000,Средний,92.916,10000.00,12.0,1114.992,1951.236,https://www.tinkoff.ru/invest/bonds/RU000A1020K7/
533,Нафтатранс Плюс выпуск 1,RU000A100303,3,10012.000000,Средний,84.303,10000.00,12.0,1011.636,252.909,https://www.tinkoff.ru/invest/bonds/RU000A100303/
623,ВЭБ.РФ ПБО-001Р-К400,RU000A104HT1,20,998.400000,Высокий,80.040,1000.00,1.0,80.040,19209.600,https://www.tinkoff.ru/invest/bonds/RU000A104HT1/
800,Селигдар Золотые облигации,RU000A1062M5,54,5609.824379,Высокий,70.992,5913.17,4.0,283.968,11500.704,https://www.tinkoff.ru/invest/bonds/RU000A1062M5/
364,РСХБ 08Т1,RU000A0JWV63,35,1037.900000,Средний,61.770,1000.00,2.0,123.540,12971.700,https://www.tinkoff.ru/invest/bonds/RU000A0JWV63/
...,...,...,...,...,...,...,...,...,...,...,...
336,Верхнебаканский цементный завод 001P выпуск 1,RU000A0ZYLK7,50,973.900000,Средний,1.566,1000.00,4.0,6.264,234.900,https://www.tinkoff.ru/invest/bonds/RU000A0ZYLK7/
349,МСБ-Лизинг 002P выпуск 2,RU000A100DC4,7,166.950120,Средний,1.479,166.80,12.0,17.748,10.353,https://www.tinkoff.ru/invest/bonds/RU000A100DC4/
681,ТД Мясничий выпуск 3,RU000A1006B5,4,145.986000,Средний,1.392,145.00,12.0,16.704,5.568,https://www.tinkoff.ru/invest/bonds/RU000A1006B5/
683,Ямало-Ненецкий АО выпуск 2,RU000A0JX0Z8,2,49.895000,Высокий,1.044,50.00,4.0,4.176,6.264,https://www.tinkoff.ru/invest/bonds/RU000A0JX0Z8/
